In [ ]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 

In [2]:
df= pd.read_excel('Online Retail.xlsx')

In [3]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [4]:
df['Description'] = df['Description'].str.strip() 
  
# Dropping the rows without any invoice number 
df.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
df['InvoiceNo'] = df['InvoiceNo'].astype('str') 
  
# Dropping all transactions which were done on credit 
df= df[~df['InvoiceNo'].str.contains('C')] 

In [5]:
# Transactions done in France 
basket_France = (df[df['Country'] =="France"] 
		.groupby(['InvoiceNo', 'Description'])['Quantity'] 
		.sum().unstack().reset_index().fillna(0) 
		.set_index('InvoiceNo')) 

# Transactions done in the United Kingdom 
basket_UK = (df[df['Country'] =="United Kingdom"] 
		.groupby(['InvoiceNo', 'Description'])['Quantity'] 
		.sum().unstack().reset_index().fillna(0) 
		.set_index('InvoiceNo')) 

# Transactions done in Portugal 
basket_Por = (df[df['Country'] =="Portugal"] 
		.groupby(['InvoiceNo', 'Description'])['Quantity'] 
		.sum().unstack().reset_index().fillna(0) 
		.set_index('InvoiceNo')) 

basket_Sweden = (df[df['Country'] =="Sweden"] 
		.groupby(['InvoiceNo', 'Description'])['Quantity'] 
		.sum().unstack().reset_index().fillna(0) 
		.set_index('InvoiceNo')) 

In [6]:
# Defining the hot encoding function to make the data suitable  
# for the concerned libraries 
def hot_encode(x): 
    if(x<= 0): 
        return False
    if(x>= 1): 
        return True
  
# Encoding the datasets 
basket_encoded = basket_France.applymap(hot_encode) 
basket_France = basket_encoded 
  
basket_encoded = basket_UK.applymap(hot_encode) 
basket_UK = basket_encoded 
  
basket_encoded = basket_Por.applymap(hot_encode) 
basket_Por = basket_encoded 
  
basket_encoded = basket_Sweden.applymap(hot_encode) 
basket_Sweden = basket_encoded 

C:\Users\ANANT TIWARI\AppData\Local\Temp\ipykernel_23028\4047986632.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded = basket_France.applymap(hot_encode)
C:\Users\ANANT TIWARI\AppData\Local\Temp\ipykernel_23028\4047986632.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded = basket_UK.applymap(hot_encode)
C:\Users\ANANT TIWARI\AppData\Local\Temp\ipykernel_23028\4047986632.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded = basket_Por.applymap(hot_encode)
C:\Users\ANANT TIWARI\AppData\Local\Temp\ipykernel_23028\4047986632.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded = basket_Sweden.applymap(hot_encode)


In [8]:
!pip install -U mlxtend

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1,num_itemsets=len(frq_items)) 

rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

Formatted Association Rules:

                                            Antecedent  \
45                        (JUMBO BAG WOODLAND ANIMALS)   
260  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
271  (PLASTERS IN TIN WOODLAND ANIMALS, RED TOADSTO...   
302  (SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...   
300  (SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...   

                          Consequent  Antecedent Support  Consequent Support  \
45                         (POSTAGE)               0.077               0.765   
260                        (POSTAGE)               0.051               0.765   
271                        (POSTAGE)               0.054               0.765   
302  (SET/6 RED SPOTTY PAPER PLATES)               0.102               0.128   
300    (SET/6 RED SPOTTY PAPER CUPS)               0.102               0.138   

     Support  Confidence  Lift  Leverage  Conviction  
45     0.077       1.000 1.307     0.018         inf  
260    0.051       1.000 1.307